## Spinning Top

Notes: 
* the code below might be slightly different than what is presented on the book. Here, many functions were created in order to promote code reusability throughout this notebook.
* the [SymPy Plotting Backends module](https://sympy-plot-backends.readthedocs.io/) and [K3D Jupyter](https://github.com/K3D-tools/K3D-jupyter) are required to visualize the results of the numerical integration in a 3D space.

In [ ]:
from sympy import *
from sympy.physics.mechanics import *
init_vprinting()

In [ ]:
phi, theta, psi = dynamicsymbols("phi, theta, psi")
phid, thetad, psid = dynamicsymbols("phi, theta, psi", 1)
omega_x, omega_y, omega_z = dynamicsymbols("omega_x omega_y omega_z")
A, C, d, m, g = symbols("A, C, d, m, g")

N, B = symbols("N, B", cls=ReferenceFrame)
B.orient_body_fixed(N, [phi, theta, psi], 313)

O, G = symbols("O, G", cls=Point)
G.set_pos(O, d * B.z)

O.set_vel(N, 0)
G.v2pt_theory(O, N, B)
G.a2pt_theory(O, N, B)

## First choice of generalized speeds and KDEs

In [ ]:
B_w_N = B.ang_vel_in(N).express(B)
B_w_N

In [ ]:
w_body = omega_x * B.x + omega_y * B.y + omega_z * B.z
rot_kin = B_w_N - w_body
rot_kin = list(rot_kin.to_matrix(B))
display(*rot_kin)

In [ ]:
top_inertia = inertia(B, A, A, C)
top = RigidBody("Top", G, B, m, (top_inertia, G))
bodies = [top]
loads = [(G, -m * g * N.z)]

kane1 = KanesMethod(
    N,
    q_ind=[psi, theta, phi],
    u_ind=[omega_x, omega_y, omega_z],
    kd_eqs=rot_kin
)
fr, frstar = kane1.kanes_equations(bodies, loads)

eom1 = fr + frstar
eom1.simplify()
eom1

In [ ]:
angle_dots = {k: v.simplify() for k, v in kane1.kindiffdict().items()}
angle_dots

In [ ]:
A_func = lambda m, h, r: 3*m * (h**2 + 4*r**2) / 80
C_func =lambda m, h, r: 3*m*r**2 / 10

r_val = 2.5e-02
h_val = r_val * 4
d_val = 3 * h_val / 4
m_val = 0.5

A_val = A_func(m_val, h_val, r_val)
C_val = C_func(m_val, h_val, r_val)

constants = {
    m: m_val, g: 9.81, d: d_val,
    A: A_val, C: C_val
}

In [ ]:
def create_func_to_integrate(kane, states, constants):
    mm = kane.mass_matrix_full
    fm = kane.forcing_full
    mm.simplify()
    fm.simplify()

    display(mm, fm)
    
    arguments = states + list(constants.keys())
    mm_func = lambdify(arguments, mm, cse=True)
    fm_func = lambdify(arguments, fm, cse=True)
    
    def func_to_integrate(t, x, args):
        values = np.concatenate((x, args))
        solution = np.linalg.solve(
            mm_func(*values),
            fm_func(*values)
        )
        return solution.flatten()
        
    return func_to_integrate

states = list(kane1.q) + list(kane1.u)
func_to_integrate = create_func_to_integrate(kane1, states, constants)

In [ ]:
from scipy.integrate import solve_ivp
import numpy as np
import time

def perform_integration(
    func_to_integrate, t0, tf, states, initial_conditions,
    constants, rtol=1e-03, atol=1e-06, method="RK45", n=None
):
    t_span = [t0, tf]
    t_eval = np.linspace(t0, tf, n) if n else n
    x0 = [float(initial_conditions[k]) for k in states]

    start_time = time.time()
    results = solve_ivp(
        func_to_integrate, t_span, x0,
        args=(list(constants.values()), ),
        method=method, t_eval=t_eval,
        rtol=rtol, atol=atol
    )
    end_time = time.time()
    print("Integration took about: %s seconds." % (end_time - start_time))
    return results

initial_conditions = {
    phi: 0, theta: pi/10, psi: 0,
    omega_x: 0, omega_y: 0, omega_z: 1000
}
results1 = perform_integration(
    func_to_integrate, 0, 2,
    states, initial_conditions, constants
)
results1

In [ ]:
import matplotlib.pyplot as plt

def plot_states(results, states, angle_dots):
    angle_dots_func = {k: lambdify(states, v) for k, v in angle_dots.items()}
    angle_dots_res = {k: v(*results.y) for k, v in angle_dots_func.items()}
    
    fig, ax = plt.subplots(3, 3, sharex=True, layout="constrained", figsize=(10, 5))
    ax[0, 0].plot(results.t, results.y.T[:, 0])
    ax[0, 0].set_ylabel("$%s$" % vlatex(states[0]))
    ax[0, 0].set_title("Angles [rad]")
    ax[1, 0].plot(results.t, results.y.T[:, 1])
    ax[1, 0].set_ylabel("$%s$" % vlatex(states[1]))
    ax[2, 0].plot(results.t, results.y.T[:, 2])
    ax[2, 0].set_ylabel("$%s$" % vlatex(states[2]))
    ax[2, 0].set_xlabel("Time [s]")
    ax[0, 1].plot(results.t, results.y.T[:, 3])
    ax[0, 1].set_ylabel("$%s$" % vlatex(states[3]))
    ax[0, 1].set_title("Velocities [rad/s]")
    ax[1, 1].plot(results.t, results.y.T[:, 4])
    ax[1, 1].set_ylabel("$%s$" % vlatex(states[4]))
    ax[2, 1].plot(results.t, results.y.T[:, 5])
    ax[2, 1].set_ylabel("$%s$" % vlatex(states[5]))
    ax[2, 1].set_xlabel("Time [s]")

    t = symbols("t")
    for i, k in enumerate(states[:3]):
        ax[i, 2].plot(results.t, angle_dots_res[k.diff(t)])
        ax[i, 2].set_ylabel("$%s$" % vlatex(k))
    plt.show()

plot_states(results1, states, angle_dots)

In [ ]:
def compuse_free_end_position(states, results):
    pos = B.z.express(N)
    pos_components = [pos & vec for vec in N]
    pos_components_funcs = [lambdify(states, comp) for comp in pos_components]
    pos_components_res = [f(*results.y) for f in pos_components_funcs]
    return pos_components_res

pos_components_res = compuse_free_end_position(states, results1)

In [ ]:
import k3d
from spb.utils import get_vertices_indices

def plot_free_end_trajectory_3d(pos_components_res, results):

    theta_num, phi_num = np.mgrid[0:2*np.pi:50j, 0:np.pi:50j]
    x = np.cos(theta_num) * np.sin(phi_num)
    y = np.sin(theta_num) * np.sin(phi_num)
    z = np.cos(phi_num)
    vertices, indices = get_vertices_indices(x, y, z)
    vertices = vertices.astype(np.float32)
    
    fig = k3d.plot(grid_visible=False)
    mesh1 = k3d.mesh(vertices, indices, side="double", opacity=0.2)
    mesh2 = k3d.mesh(vertices, indices, side="double", opacity=0.025, color=0, wireframe=True)
    line_coordinates = np.vstack(pos_components_res).T.astype(np.float32)
    path = k3d.line(
        line_coordinates,
        shader="simple",
        attribute=results.t
    )
    fig += mesh1
    fig += mesh2
    fig += path
    fig.display()

plot_free_end_trajectory_3d(pos_components_res, results1)

In [ ]:
from matplotlib.collections import LineCollection

# from Matplotlib's documentation:
# https://matplotlib.org/stable/gallery/lines_bars_and_markers/multicolored_line.html
def colored_line(x, y, c, ax, **lc_kwargs):
    """
    Plot a line with a color specified along the line by a third value.

    It does this by creating a collection of line segments. Each line segment is
    made up of two straight lines each connecting the current (x, y) point to the
    midpoints of the lines connecting the current point with its two neighbors.
    This creates a smooth line with no gaps between the line segments.

    Parameters
    ----------
    x, y : array-like
        The horizontal and vertical coordinates of the data points.
    c : array-like
        The color values, which should be the same size as x and y.
    ax : Axes
        Axis object on which to plot the colored line.
    **lc_kwargs
        Any additional arguments to pass to matplotlib.collections.LineCollection
        constructor. This should not include the array keyword argument because
        that is set to the color argument. If provided, it will be overridden.

    Returns
    -------
    matplotlib.collections.LineCollection
        The generated line collection representing the colored line.
    """
    if "array" in lc_kwargs:
        warnings.warn('The provided "array" keyword argument will be overridden')

    # Default the capstyle to butt so that the line segments smoothly line up
    default_kwargs = {"capstyle": "butt"}
    default_kwargs.update(lc_kwargs)

    # Compute the midpoints of the line segments. Include the first and last points
    # twice so we don't need any special syntax later to handle them.
    x = np.asarray(x)
    y = np.asarray(y)
    x_midpts = np.hstack((x[0], 0.5 * (x[1:] + x[:-1]), x[-1]))
    y_midpts = np.hstack((y[0], 0.5 * (y[1:] + y[:-1]), y[-1]))

    # Determine the start, middle, and end coordinate pair of each line segment.
    # Use the reshape to add an extra dimension so each pair of points is in its
    # own list. Then concatenate them to create:
    # [
    #   [(x1_start, y1_start), (x1_mid, y1_mid), (x1_end, y1_end)],
    #   [(x2_start, y2_start), (x2_mid, y2_mid), (x2_end, y2_end)],
    #   ...
    # ]
    coord_start = np.column_stack((x_midpts[:-1], y_midpts[:-1]))[:, np.newaxis, :]
    coord_mid = np.column_stack((x, y))[:, np.newaxis, :]
    coord_end = np.column_stack((x_midpts[1:], y_midpts[1:]))[:, np.newaxis, :]
    segments = np.concatenate((coord_start, coord_mid, coord_end), axis=1)

    lc = LineCollection(segments, **default_kwargs)
    lc.set_array(c)  # set the colors of each segment

    return ax.add_collection(lc)

def cart2sph(x, y, z):
    # convert cartesian coordinates to spherical coordinates
    xy = np.sqrt(x**2 + y**2)
    r = np.sqrt(x**2 + y**2 + z**2)
    theta = np.arctan2(y, x) 
    phi = np.arctan2(xy, z) 
    return r, theta, phi

def plot_free_end_trajectory_2d(pos_components_res, results):
    # t: theta (longitude), p: phi for spherical to cylindrical projection.
    # Not to be confused with theta and phi used in the generation of EoM
    _, t, p = cart2sph(*pos_components_res)
    t, p = [np.degrees(e) for e in [t, p]]
    t_with_nan = t.copy()
    
    # t ranges from -180deg to +180deg. As t cross over +180deg and start
    # the loop again at -180deg, Matplotlib will draw an horizontal line,
    # which is not desired. Let's remove it by inserting a NaN value
    # where this discontinuity happens.
    for i in range(1, len(t)):
        if abs(t[i] - t[i-1]) > 90:
            t_with_nan[i] = np.nan
    
    fig, ax = plt.subplots(figsize=(10, 5))
    cl = colored_line(t_with_nan, 90 - p, results.t, ax, cmap="plasma")
    fig.colorbar(cl, label="Time [s]")
    ax.set_xlabel(r"$\theta$ [deg]")
    ax.set_ylabel(r"$\phi$ [deg]")
    ax.set_xlim(-180, 180)
    ax.set_ylim(55, 80)
    ax.set_title("Drift ")
    plt.show()

plot_free_end_trajectory_2d(pos_components_res, results1)

In [ ]:
def compute_and_plot_energies(kane, states, constants, results):
    vG = G.vel(N)
    T = (B_w_N & top_inertia & B_w_N) / 2 + m * (vG & vG) / 2
    T = T.subs(kane.kindiffdict()).simplify().doit()
    U = m * g * (G.pos_from(O) & N.z)
    display(T, U)

    arguments = states + list(constants.keys())
    T_func = lambdify(arguments, T)
    U_func = lambdify(arguments, U)
    T_res = T_func(*results.y, *list(constants.values()))
    U_res = U_func(*results.y, *list(constants.values()))
    
    fig, ax = plt.subplots(3, 1, sharex=True)
    ax[0].plot(results.t, T_res)
    ax[0].set_ylabel("T [J]")
    ax[1].plot(results.t, U_res)
    ax[1].set_ylabel("U [J]")
    ax[2].plot(results.t, T_res + U_res)
    ax[2].set_xlabel("Times [s]")
    ax[2].set_ylabel("T+U [J]")
    fig.suptitle("Check for Energy Conservation")
    plt.show()

    return T_res, U_res

compute_and_plot_energies(kane1, states, constants, results1);

### Improving accuracy of the results

In [ ]:
results2 = perform_integration(
    func_to_integrate, 0, 2,
    states, initial_conditions, constants,
    rtol=1e-06, atol=1e-09
)
pos_components_res2 = compuse_free_end_position(states, results2)
plot_free_end_trajectory_3d(pos_components_res2, results2)
plot_free_end_trajectory_2d(pos_components_res2, results2)
T_res, U_res = compute_and_plot_energies(kane1, states, constants, results2)

## Second choice of generalized speeds and KDEs

In [ ]:
omega_1, omega_2, omega_3 = dynamicsymbols("omega_1, omega_2, omega_3")
kd_eqs = [
    omega_1 - phid, omega_2 - thetad, omega_3 - psid
]

kane3 = KanesMethod(
    N,
    q_ind=[psi, theta, phi],
    u_ind=[omega_1, omega_2, omega_3],
    kd_eqs=kd_eqs
)
fr, frstar = kane3.kanes_equations(bodies, loads)

eom3 = fr + frstar
eom3.simplify()
eom3

In [ ]:
states3 = list(kane3.q) + list(kane3.u)
func_to_integrate3 = create_func_to_integrate(kane3, states3, constants)

initial_conditions3 = {
    phi: 0, theta: pi/10, psi: 0,
    omega_1: 0, omega_2: 0, omega_3: 1000
}

results3 = perform_integration(
    func_to_integrate3, 0, 2,
    states3, initial_conditions3, constants,
    method="RK45", n=2000,
    # rtol=1e-06, atol=1e-09
)
pos_components_res3 = compuse_free_end_position(states3, results3)
plot_free_end_trajectory_3d(pos_components_res3, results3)
plot_free_end_trajectory_2d(pos_components_res3, results3)
T_res3, U_res3 = compute_and_plot_energies(kane3, states3, constants, results3)